In [1]:
import pandas as pd
from src.infrastructure.database.client import PostgresClient
from src.infrastructure.config.settings import settings

from src.infrastructure.database.repositories.symbol_strategy_repository import SymbolStrategyRepository
from src.domain.signals.signals_service import SignalService
from src.domain.signals.aggregator import SignalAggregator

from src.infrastructure.data.yfinance import YFinanceService
from src.domain.etl.services.pipeline_service import FeaturePipelineService
from src.domain.etl.services.extractor_service import ExtractorService
from src.domain.etl.services.enricher_service import EnricherService
from src.infrastructure.broker.broker_fake import FakeBrokerClient

# --- Init DB ---
db_client = PostgresClient(
    db_url=settings.db_url,
    pool_size=settings.db_pool_size,
    max_overflow=settings.db_max_overflow,
    pool_timeout=settings.db_pool_timeout,
    pool_recycle=settings.db_pool_recycle,
    echo=settings.db_echo,
)
await db_client.init(create_schema=True)

aggregator = SignalAggregator()
signal_service = SignalService(db_client, aggregator)


# --- Load Data ---
yfs = YFinanceService()
broker = FakeBrokerClient(db_client=db_client, price_provider=yfs)
extractor = ExtractorService(yfs)
enricher = EnricherService()
pipeline = FeaturePipelineService(extractor, enricher)

# define start and end for the last 100 candles for 1h timeframe
start = pd.Timestamp.now(tz="UTC") - pd.Timedelta(hours=500)
end = pd.Timestamp.now(tz="UTC")

# define start and end for the last 100 candles for 1d timeframe
# start = pd.Timestamp.now(tz="UTC") - pd.Timedelta(days=200)
# end = pd.Timestamp.now(tz="UTC")
for symbol in yfs.DEFAULT_CEDEARS:
    data = await pipeline.start(symbols=symbol, timeframe="1h", start=start, end=end)

    df = data.ohlcv

    # --- Generate Signal ---
    signal = await signal_service.generate_for_symbol(
        symbol=symbol,
        df=df,
        timeframe="1h",
    )

    if signal:
        print("------") 
        print("Id=", signal.id)
        print("Symbol=", signal.symbol)
        print("Signal Source=", signal.signal_source)
        print("Signal Type=", signal.signal_type)
        print("Confidence=", signal.confidence)
        print("Strength=", signal.strength)
        print("Price=", signal.price)
        print("Target Price=", signal.target_price)
        print("Stop Loss=", signal.stop_loss)
        print("Meta=", signal.meta)
        print("Created At=", signal.created_at)
        print("Expires At=", signal.expires_at)

        from src.domain.portfolio.optimizers.hrp_optimizer import HRPOptimizer
        from src.domain.portfolio.portfolio_service import PortfolioService

        optimizer = HRPOptimizer()
        portfolio_service = PortfolioService(db_client=db_client, extractor=yfs, optimizer=optimizer, broker=broker)

        order_size = await portfolio_service.compute_order_size(signal=signal)
        print("Computed Order Size=", order_size)
        print("------") 



INFO:src.domain.etl.services.enricher_service:✅ BreakoutAnalyzer completed
INFO:src.domain.etl.services.enricher_service:✅ Stage 2 completed: 3 analyzers applied
INFO:src.domain.etl.services.enricher_service:🧹 Stage 3: Validation and cleanup
INFO:src.domain.etl.services.enricher_service:✅ Stage 3 completed
INFO:src.domain.etl.services.enricher_service:✅ TRANSFORM completed: 6→63 columns
INFO:src.domain.signals.signals_service:🎯 Generating signal using strategy: MACDStrategy
INFO:src.domain.signals.interpreters.structure_interpreter:🏗️ Analyzing market structure...
INFO:src.domain.signals.interpreters.volume_interpreter:📦 Analizando volume signals...
INFO:src.domain.signals.interpreters.trend_interpreter:📊 Analizando trend signals...
INFO:src.domain.signals.interpreters.momentum_interpreter:📈 Analizando momentum signals...
INFO:src.domain.signals.interpreters.momentum_interpreter:✅ Momentum analysis: 1 signals detectadas
INFO:src.domain.signals.interpreters.risk_interpreter:⚠️ Evaluatin

------
Id= None
Symbol= AAPL.BA
Signal Source= SignalSourceEnum.MOMENTUM
Signal Type= SignalTypeEnum.HOLD
Confidence= 1.0
Strength= SignalStrengthEnum.WEAK
Price= 20880.0
Target Price= None
Stop Loss= None
Meta= {'kind': 'momentum', 'indicator': 'STOCHRSIk_14_14_3_3', 'condition': 'overbought', 'value': 56.547, 'strategy_name': 'MACDStrategy', 'aggregated': True, 'source_signals_count': 1, 'source_types': {'sell': 1}, 'source_sources': {'momentum': 1}, 'aggregator_info': {'chosen_type': 'hold', 'final_score': 0.275}, 'risk_context': {'has_risk_alerts': False, 'high_volatility': False, 'volume_spike': False, 'risk_events': False, 'extreme_conditions': False, 'risk_alerts_count': 0}}
Created At= 2025-12-10 10:31:44.222799
Expires At= 2026-06-08 10:31:44.221763


INFO:src.domain.portfolio.portfolio_service:HOLD signal for AAPL.BA, no operation.


Computed Order Size= 0.0
------


INFO:src.domain.etl.services.extractor_service:📥 Extracted 78 rows for symbols=['TSLA.BA']
INFO:src.domain.etl.services.enricher_service:🔧 TRANSFORM: Starting transformation pipeline, for the next indicators: 
INFO:src.domain.etl.services.enricher_service:📊 Stage 1: Applying mathematical indicators (pandas-ta)
INFO:src.domain.etl.calculators.basic_indicators_calculator:🚀 IndicatorAnalyzer starting with 78 records
INFO:src.domain.etl.calculators.basic_indicators_calculator:🔍 No indicators specified, using all
INFO:src.domain.etl.calculators.basic_indicators_calculator:📈 Calculating momentum indicators...
INFO:src.domain.etl.calculators.basic_indicators_calculator:📊 Calculating trend indicators...
INFO:src.domain.etl.calculators.basic_indicators_calculator:📦 Calculating volume indicators...
INFO:src.domain.etl.calculators.basic_indicators_calculator:✅ Volume spike calculated
INFO:src.domain.etl.calculators.basic_indicators_calculator:⚡ Calculating volatility indicators...
INFO:src.domain

...

INFO:src.domain.etl.services.extractor_service:📥 Extracted 78 rows for symbols=['MELI.BA']
INFO:src.domain.etl.services.enricher_service:🔧 TRANSFORM: Starting transformation pipeline, for the next indicators: 
INFO:src.domain.etl.services.enricher_service:📊 Stage 1: Applying mathematical indicators (pandas-ta)
INFO:src.domain.etl.calculators.basic_indicators_calculator:🚀 IndicatorAnalyzer starting with 78 records
INFO:src.domain.etl.calculators.basic_indicators_calculator:🔍 No indicators specified, using all
INFO:src.domain.etl.calculators.basic_indicators_calculator:📈 Calculating momentum indicators...
INFO:src.domain.etl.calculators.basic_indicators_calculator:📊 Calculating trend indicators...
INFO:src.domain.etl.calculators.basic_indicators_calculator:📦 Calculating volume indicators...
INFO:src.domain.etl.calculators.basic_indicators_calculator:✅ Volume spike calculated
INFO:src.domain.etl.calculators.basic_indicators_calculator:⚡ Calculating volatility indicators...
INFO:src.domain

------
Id= None
Symbol= MELI.BA
Signal Source= SignalSourceEnum.MOMENTUM
Signal Type= SignalTypeEnum.HOLD
Confidence= 1.0
Strength= SignalStrengthEnum.WEAK
Price= 25800.0
Target Price= None
Stop Loss= None
Meta= {'kind': 'momentum', 'indicator': 'STOCHRSIk_14_14_3_3', 'condition': 'overbought', 'value': 16.053, 'strategy_name': 'RSIStrategy', 'aggregated': True, 'source_signals_count': 2, 'source_types': {'buy': 1, 'sell': 1}, 'source_sources': {'momentum': 2}, 'aggregator_info': {'chosen_type': 'hold', 'final_score': 0.275}, 'risk_context': {'has_risk_alerts': False, 'high_volatility': False, 'volume_spike': False, 'risk_events': False, 'extreme_conditions': False, 'risk_alerts_count': 0}}
Created At= 2025-12-10 10:31:46.675318
Expires At= 2026-06-08 10:31:46.674383
Computed Order Size= 0.0
------


INFO:src.domain.etl.services.extractor_service:📥 Extracted 78 rows for symbols=['SPY.BA']
INFO:src.domain.etl.services.enricher_service:🔧 TRANSFORM: Starting transformation pipeline, for the next indicators: 
INFO:src.domain.etl.services.enricher_service:📊 Stage 1: Applying mathematical indicators (pandas-ta)
INFO:src.domain.etl.calculators.basic_indicators_calculator:🚀 IndicatorAnalyzer starting with 78 records
INFO:src.domain.etl.calculators.basic_indicators_calculator:🔍 No indicators specified, using all
INFO:src.domain.etl.calculators.basic_indicators_calculator:📈 Calculating momentum indicators...
INFO:src.domain.etl.calculators.basic_indicators_calculator:📊 Calculating trend indicators...
INFO:src.domain.etl.calculators.basic_indicators_calculator:📦 Calculating volume indicators...
INFO:src.domain.etl.calculators.basic_indicators_calculator:✅ Volume spike calculated
INFO:src.domain.etl.calculators.basic_indicators_calculator:⚡ Calculating volatility indicators...
INFO:src.domain.

------
Id= None
Symbol= SPY.BA
Signal Source= SignalSourceEnum.MOMENTUM
Signal Type= SignalTypeEnum.HOLD
Confidence= 1.0
Strength= SignalStrengthEnum.WEAK
Price= 51425.0
Target Price= None
Stop Loss= None
Meta= {'kind': 'momentum', 'indicator': 'STOCHRSIk_14_14_3_3', 'condition': 'overbought', 'value': 24.81, 'strategy_name': 'SMACrossoverWithRSIAndAvgVolStrategy', 'aggregated': True, 'source_signals_count': 1, 'source_types': {'sell': 1}, 'source_sources': {'momentum': 1}, 'aggregator_info': {'chosen_type': 'hold', 'final_score': 0.275}, 'risk_context': {'has_risk_alerts': False, 'high_volatility': False, 'volume_spike': False, 'risk_events': False, 'extreme_conditions': False, 'risk_alerts_count': 0}}
Created At= 2025-12-10 10:31:47.059312
Expires At= 2026-06-08 10:31:47.058416
Computed Order Size= 0.0
------


INFO:src.domain.etl.services.extractor_service:📥 Extracted 78 rows for symbols=['QQQ.BA']
INFO:src.domain.etl.services.enricher_service:🔧 TRANSFORM: Starting transformation pipeline, for the next indicators: 
INFO:src.domain.etl.services.enricher_service:📊 Stage 1: Applying mathematical indicators (pandas-ta)
INFO:src.domain.etl.calculators.basic_indicators_calculator:🚀 IndicatorAnalyzer starting with 78 records
INFO:src.domain.etl.calculators.basic_indicators_calculator:🔍 No indicators specified, using all
INFO:src.domain.etl.calculators.basic_indicators_calculator:📈 Calculating momentum indicators...
INFO:src.domain.etl.calculators.basic_indicators_calculator:📊 Calculating trend indicators...
INFO:src.domain.etl.calculators.basic_indicators_calculator:📦 Calculating volume indicators...
INFO:src.domain.etl.calculators.basic_indicators_calculator:✅ Volume spike calculated
INFO:src.domain.etl.calculators.basic_indicators_calculator:⚡ Calculating volatility indicators...
INFO:src.domain.